In [3]:
data_path = "/data1/CLERMS/data/"

In [4]:
import os
import sys
import numpy as np
from tqdm.notebook import tqdm
from matchms.importing import load_from_json
import pickle
from matchms.importing import load_from_mgf

with open(data_path + "train_1.pickle", 'rb') as f:
    train1 = pickle.load(f)
with open(data_path +"train_2.pickle", 'rb') as f:
    train2 = pickle.load(f)
with open(data_path +"test_1.pickle", 'rb') as f:
    test1 = pickle.load(f)
with open(data_path +"test_2.pickle", 'rb') as f:
    test2 = pickle.load(f)
    
spectrums = train1 + train2 + test1 + test2
# spectrums = list(load_from_mgf(spectrums_filepath))
print("number of spectra:", len(spectrums))

from matchms.filtering import normalize_intensities
from matchms.filtering import require_minimum_number_of_peaks
from matchms.filtering import select_by_mz
from matchms.filtering import select_by_relative_intensity
from matchms.filtering import reduce_to_number_of_peaks
from matchms.filtering import add_losses

number of spectra: 109734


In [3]:
spectrums = [s for s in spectrums if s is not None]
print("Number of remaining spectra:", len(spectrums))

number_of_peaks = [len(spec.peaks) for spec in spectrums]
print("Maximum number of peaks in one spectrum:", np.max(number_of_peaks))
print("Number of spectra with > 1000 peaks:", np.sum(np.array(number_of_peaks) > 1000))
print("Number of spectra with > 2000 peaks:", np.sum(np.array(number_of_peaks) > 2000))
print("Number of spectra with > 5000 peaks:", np.sum(np.array(number_of_peaks) > 5000))
print("Careful: Number of spectra with < 10 peaks:", np.sum(np.array(number_of_peaks) < 10))

ID = 2
if spectrums[ID].get("inchi") + spectrums[ID].get("smiles"):
    print(spectrums[ID].get("inchi") + "\n\n" + spectrums[ID].get("smiles"))

Number of remaining spectra: 109734
Maximum number of peaks in one spectrum: 37922
Number of spectra with > 1000 peaks: 5216
Number of spectra with > 2000 peaks: 2099
Number of spectra with > 5000 peaks: 639
Careful: Number of spectra with < 10 peaks: 13422
InChI=1S/C6H7O4P/c7-11(8,9)10-6-4-2-1-3-5-6/h1-5H,(H2,7,8,9)

c(c1)ccc(c1)OP(O)(O)=O


In [4]:
def annotated(s):
    return (s.get("inchi") or s.get("smiles")) and s.get("inchikey")


annotation_list = []
for i, s in enumerate(spectrums):
    if annotated(s):
        annotation_list.append((i, s.get("inchi"), s.get("smiles"), s.get("inchikey")))
print(f"Unique inchikeys (14char) in annotated dat: {len({x[3][:14] for x in annotation_list})}")
spectrums_annotated = [s for s in spectrums if annotated(s)]
print("useful spectra nums：", len(spectrums_annotated), "all spectra nums", len(spectrums))

Unique inchikeys (14char) in annotated dat: 15062
有用的质谱数量： 109734 总的数量是 109734


In [5]:
from collections import Counter


def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]


inchikeys_list = []
for s in spectrums_annotated:
    inchikeys_list.append(s.get("inchikey"))

inchikeys14_array = np.array([x[:14] for x in inchikeys_list])

inchikeys14_unique = list({x[:14] for x in inchikeys_list})

inchikey14 = inchikeys14_unique[12]
print(inchikey14)

idx = np.where(inchikeys14_array == inchikey14)[0]
for i in idx:
    print(spectrums_annotated[i].get("smiles") + "\n")

print("most frequent:", most_frequent([spectrums_annotated[i].get("smiles") for i in idx]))

inchi_list = []
for s in spectrums_annotated:
    inchi_list.append(s.get("inchi"))

HCEYJYMNIQHPPK
CC1(C)C(=O)C=C[C@@]2(C)[C@H]1C[C@@H](O)[C@]1(C)[C@@H]2CC[C@@]2(C)[C@H](c3ccoc3)OC(=O)[C@H]3O[C@]321

CC1(C)C(=O)C=C[C@@]2(C)[C@H]1C[C@@H](O)[C@]1(C)[C@@H]2CC[C@@]2(C)[C@H](c3ccoc3)OC(=O)[C@H]3O[C@]321

CC1(C)C(=O)C=C[C@@]2(C)[C@H]1C[C@@H](O)[C@]1(C)[C@@H]2CC[C@@]2(C)[C@H](c3ccoc3)OC(=O)[C@H]3O[C@]321

CC1(C)C(=O)C=C[C@@]2(C)[C@H]1C[C@@H](O)[C@]1(C)[C@@H]2CC[C@@]2(C)[C@H](c3ccoc3)OC(=O)[C@H]3O[C@]321

CC1(C)C(=O)C=C[C@@]2(C)[C@H]1C[C@@H](O)[C@]1(C)[C@@H]2CC[C@@]2(C)[C@H](c3ccoc3)OC(=O)[C@H]3O[C@]321

CC1(C)C(=O)C=C[C@@]2(C)[C@H]1C[C@@H](O)[C@]1(C)[C@@H]2CC[C@@]2(C)[C@H](c3ccoc3)OC(=O)[C@H]3O[C@]321

CC1(C)C(=O)C=C[C@@]2(C)[C@H]1C[C@@H](O)[C@]1(C)[C@@H]2CC[C@@]2(C)[C@H](c3ccoc3)OC(=O)[C@H]3O[C@]321

CC1(C)C(=O)C=C[C@@]2(C)[C@H]1C[C@@H](O)[C@]1(C)[C@@H]2CC[C@@]2(C)[C@H](c3ccoc3)OC(=O)[C@H]3O[C@]321

most frequent: CC1(C)C(=O)C=C[C@@]2(C)[C@H]1C[C@@H](O)[C@]1(C)[C@@H]2CC[C@@]2(C)[C@H](c3ccoc3)OC(=O)[C@H]3O[C@]321


In [6]:
inchi_array = np.array(inchi_list)

inchi_mapping = []
ID_mapping = []
inchikeys_unique = []
for inchikey14 in inchikeys14_unique:
    idx = np.where(inchikeys14_array == inchikey14)[0]
    inchi = most_frequent([spectrums_annotated[i].get("inchi") for i in idx])
    inchi_mapping.append(inchi)
    ID = idx[np.where(inchi_array[idx] == inchi)[0][0]]
    ID_mapping.append(ID)

    indd = inchikeys14_array[idx][0]
    inchikeys_unique.append(indd)
# In[47]:


import pandas as pd

metadata = pd.DataFrame(list(zip(inchikeys_unique, inchi_mapping, ID_mapping)), columns=["inchikey", "inchi", "ID"])

from matchms.filtering.add_fingerprint import add_fingerprint

for i in tqdm(metadata.ID.values):
    spectrums_annotated[i] = add_fingerprint(spectrums_annotated[i], fingerprint_type="daylight", nbits=2048)

[09:28:38] WARNING: not removing hydrogen atom without neighbors


In [ ]:
for i in tqdm(metadata.ID.values):
    if np.any(np.isnan(spectrums_annotated[i].get("fingerprint"))):
        print(i)
from matchms.similarity import FingerprintSimilarity

spectrums_represent = [spectrums_annotated[i] for i in metadata.ID.values]

similarity_measure = FingerprintSimilarity(similarity_measure="jaccard")
import numpy


def jaccard_index(u, v):
    u_or_v = numpy.bitwise_or(u != 0, v != 0)
    u_and_v = numpy.bitwise_and(u != 0, v != 0)
    jaccard_score = 0
    if u_or_v.sum() != 0:
        jaccard_score = numpy.float64(u_and_v.sum()) / numpy.float64(u_or_v.sum())
    return jaccard_score


scores_mol_similarity = similarity_measure.matrix(spectrums_represent, spectrums_represent)
tanimoto_df = pd.DataFrame(scores_mol_similarity, columns=metadata.inchikey.values, index=metadata.inchikey.values)

In [8]:
with open("/data1/CLERMS/data/ALL_GNPS_210125_positive_daylight_tanimoto_scores.pkl",'wb') as f:
    pickle.dump(tanimoto_df, f)